In [1]:
# Necessary imports of functions and dictionaries
# from dictionaries import *
# from sample_sheet_functions import *
import pandas as pd
from io import StringIO
from base64 import b64decode
import re


# Specific imports to clean up files
from dictionaries import different_24_dict, different_48_dict, different_96_dict, complete_sample_sheet_string, make_hiseq_header, make_miniseq_header, make_miseq_header, make_nextseq_header, f_primers_to_reverse_complement
from dash_sample_sheet_functions import *


###### For future
- Give possibility to download/upload in CSV/XLSX
- Table shows string on tooltip
- Table captures with a regular expression the sample name prefixes


# User download for template
_User Required to_
- Pick the table flavours they want (in form of dictionary key:True/False)
- Add project name, experiment name and additional comments (Now all stored in sample sheet template)

In [2]:
# Essential input
plates_options = {'24': False, '48': False, 'e': False, 'f': False, 'g': False, 'h': False, 'i': False, 'j': False, 'k': False, 'l': False}
project_name = ''

# Non-essential
experiment_name = ''
additional_comments = ''

file_format=None #Needs to be done at some point

In [3]:
# User input simulation
user_project_name = 'user project'
user_experiment_name = 'this is my project'
user_additional_comments = 'i want more space for the comments!'

user_plate_selection = {'e': False, 'f': True, 'g': False, 'h': False, 'i': False, 'j': False, 'k': True, 'l': True, 'tf': False, 'fe': True}


In [4]:
#### !!!!!!
def wrapper_make_template(selection, project_name, experiment_name, additional_comments, file_format='csv'):
    sample_sheet = ''
    filename = ''

    # Iterate through the selection, if a key is true, add it to the sample sheet.
    # If the key is tf (24) or fe (48), handle it accordingle

    for flavour, value in selection.items():
        # If the selection is false, continue
        if not value:
            continue
        # If the key is 24/48 act accordingly
        if flavour == 'tf':
            import_df = pd.DataFrame.from_dict(different_24_dict)
            string = import_df.to_csv(index=False)
            
        elif flavour == 'fe':
            import_df = pd.DataFrame.from_dict(different_48_dict)
            string = import_df.to_csv(index=False)
        
        else:
            # Standard 96 well plate, add flavour to it
            import_df = pd.DataFrame.from_dict(different_96_dict)
            new_cols = [x if not x.startswith('Unnamed') else flavour for x in import_df.columns]
            import_df.columns = new_cols

            string = import_df.to_csv(index=False)

        #Append string to sample sheet collective string + new line
        sample_sheet += string + ',,,,,,,,,,,,\n'


    # Add metadata to sample sheet
    metadata = f"""Project Name,{project_name},,,,,,,,,,,\nExperiment Name,{experiment_name},,,,,,,,,,,\nComments,{additional_comments},,,,,,,,,,,\n"""
    sample_sheet = metadata + sample_sheet

    project_name = project_name.replace(' ', '_')
    filename = f'template_{project_name}.{file_format}'

    return filename, sample_sheet


In [5]:
filename, sample_sheet = wrapper_make_template(user_plate_selection, user_project_name, user_experiment_name, user_additional_comments)

In [6]:
with open('delete_now.csv', 'w') as file:
    file.write(sample_sheet)


# User uploads template

_Persisting from before:_
- Project name
- Experiment name
- Comments 
- Flavours

_Requires user_ 
- Pick an instrument and the type of sample sheet required!

_In background_
- Sanity checks eg 
    - Checks number of samples
    - Checks number of blanks/controls
    - Checks no samples start with underscore (_)

_Processes:_
- Read file in, extract different tables with flavours and return it for it to display
- Pre process complete sample sheet?
- Get user selection and produce sample



In [7]:
# Obtain information from the upload file and extract tables, reports and metadata
def wrapper_handle_upload(content, filename):
    #Checks before the function starts for filename and content
    if filename is not None and filename.split('.')[1] != 'csv':
        #return genericLabelWrapper("Please only upload 'csv' files"), None
        return "Please only upload 'csv' files", None, None
    if content is None:
        # return genericLabelWrapper('The content of the file you uploaded is not valid'), None
        return 'The content of the file you uploaded is not valid', None, None
        
    
    # Handle file
    # UNCOMMENT THIS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # content_type, content_string = content.split(',')
    # decoded_content = b64decode(content_string)
    
    # file_content = StringIO(decoded_content.decode('utf-8')).readlines()
    file_content = content


    # Slice metadata eg first 3 lines
    metadata = file_content[:3]
    # convert metadata
    metadata_dict = get_metadata(metadata)

    
    # Get rest of file, read it in with pandas and get the tables
    file_string = ''.join(file_content[3:])
    # Read file in with pandas
    df = pd.read_csv(StringIO(file_string), header=None)
    # Helper function to extract the tables and return it in dictionary where flavour:df
    tables = get_tables(df)

    # Perform checks on tables
    checks_tables = {key:check_table(value) for key, value in tables.items()}

    return tables, checks_tables, metadata_dict



In [8]:
filename = 'delete_now.csv'

# Read file in 
with open(filename, 'r') as file:
    file_content = file.readlines()


In [9]:
#!!!!!!!!!!!!!!!!!!!!!!!!
tables, reports, metadata_dict = wrapper_handle_upload(file_content, filename)

# Final step, prepare sample sheet based on requested machinery/format and submit!

_Required from user_
- Machinery selection
- Option to download

_Steps_:
- Get user input
- Get user's tables and metadata
- Melt everything in wrapper function that returns filename and file content


NEED TO ADD SOMETHING TO APPEND THE 24/48 well plates

In [10]:
# Data recouped from before 
metadata_dict = metadata_dict
tables = tables
project_name = metadata_dict['project_name']
experiment_name = metadata_dict['experiment_name']
comments = metadata_dict['comments']

In [11]:
# User selections
user_machinery_selection = 'mi_seq'
user_sheet_complete = True


In [12]:
def add_tf_fe(sample_sheet, table):
    flavour, table = table

    print(flavour, table)






#"Sample_ID","Sample_Name","Sample_Plate","Sample_Well","I7_Index_ID","index","I5_Index_ID","index2","Sample_Project","Description"
#1,sample,F,A01,R936,TGGAACGCT,F904,CTAATGCAC,,


In [13]:
# Start wrapper

# Get the complete sample sheet
def wrapper_create_complete_sample_sheet(tables, complete, machinery, project, experiment, comments):
    # Handle the presence of 24/48 plates
    tf_or_fe = None
    # Retain 24/48 well layouts and produce the sample sheet for the rest
    if 'tf' in tables.keys():
        tf_or_fe = ('tf', tables['tf']) 
        del tables['tf']
    if 'fe' in tables.keys():
        tf_or_fe = ('fe', tables['fe']) 
        del tables['fe']

    # Get body of sample sheet
    sample_sheet_body = get_sample_sheet_body(tables, complete)

    # Append 24/48 plate to it
    # if tf_or_fe:
    #     sample_sheet_body = add_tf_fe(sample_sheet_body, tf_or_fe)



    # Make header and adapt file based on machine chosen
    header = None
    filename=None
    # 
    if machinery == 'hi_seq':
        # The header is basically empty
        header = make_hiseq_header()
        filename = f'{project}_SR_HS{"_all_primers" if complete else ""}.csv'
        # File does not change
    #
    elif machinery == 'mini_seq':
        # Header only has experiment 
        header = make_miniseq_header(experiment)
        # Refactor the columns to reduce them to the required number
        sheet = sheet[['Sample_ID', 'Description', 'I7_Index_ID', 'index', 'I5_Index_ID', 'index2', 'Sample_Project']]
        filename = f'{project}_MiniSeq{"_all_primers" if complete else ""}.csv'
    #
    elif machinery == 'mi_seq':
        # Nothing special
        header = make_miseq_header(project, experiment, comments)
        filename = f'{project}_SR_MS{"_all_primers" if complete else ""}.csv'
    #
    elif machinery == 'next_seq':
        header = make_nextseq_header(project, experiment, comments)
        # Next seq requires reverse I5, map the index names to the correct sequence from the dictionary
        sheet['index2'] = sheet['I5_Index_ID'].map(f_primers_to_reverse_complement)
        filename = f'{project}_PE_NS{"_all_primers" if complete else ""}.csv'
    else:
        raise Exception('I do not recognise this machinery instrument!')

    text = header + sample_sheet_body.to_csv(index=False, header=False)

    return filename, text




    


In [15]:
filename,sample_sheet = wrapper_create_complete_sample_sheet(tables, user_sheet_complete, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments)

In [ ]:
with open('sample_sheet', 'w') as file:
    file.write(sample_sheet)

In [ ]:
# LEFT TO DO
# ADD THE 24/48 (Complete function)
# Perform checks to see if everything is correct
# Implement in app